**Installing & Importing Required Libraries**

In [ ]:
!pip install transformers

In [ ]:
import re
import json
import h5py
import tqdm
import numpy as np
import pandas as pd
import joblib as jb
from transformers import GPT2Tokenizer
from sklearn.model_selection import train_test_split

**Mounting Google Drive for importing the Data Files which will be used in the Tokenization**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Importing Updated Recipe json File which contains Recipe Data**

In [ ]:
df_new = jb.load('/content/drive/MyDrive/BTP_Dev/Dataset/data_v1.pickle')

**Converting Recipe json File into a DataFrame**

In [ ]:
df = pd.DataFrame(df_new)

In [ ]:
df.head()

**Formatting the Instructions of the Recipe by Performing operations like removing '\t' from the beginning of the instructions, inserting ';' at the end of each instruction, etc**

In [ ]:
list_of_instrns = []
for row in range(0,len(df)):
    instr = df.iloc[row]['instructions']

    strg = ""
    length = len(instr) - 1
    count = 0
    for instruction in instr:
        processed_instr = []
        for j in range(0,len(instruction)):
            if(instruction[j]=='|' or instruction[j]=='\t'):
                continue
            elif(instruction[j]==' '):
                if(instruction[j-1]!='|'):
                   strg = strg + instruction[j]
            elif(instruction[j] == '.') and (j!=len(instruction)-1) and (instruction[j-1].isdigit()==False):
                strg = strg + ' '
                strg = strg + instruction[j]
            elif(instruction[j]>='a' and instruction[j]<='z') or (instruction[j]>='A' and instruction[j]<='Z') :
                 strg =  strg + instruction[j].lower()
            elif(instruction[j] == ','):
                  strg =  strg + ' '
                  strg =  strg + ','
            elif(instruction[j].isdigit()):
                if(instruction[j+1] == '.') or (instruction[j+2] == '.'):
                    continue
                else:
                    strg = strg + instruction[j]

        if(count!=length):
            strg = strg + ' '
            strg = strg + ';'
            strg = strg + ' '


        count = count + 1

    processed_instr.append(strg)
    list_of_instrns.append(processed_instr)

**Deleting the current "instructions" column from the DataFrame and inserting the modified Instructions by Creating the new "instructions" column**

In [ ]:
df.drop('instructions', inplace=True, axis=1)

In [ ]:
df['instructions'] = list_of_instrns

In [ ]:
df.head()

**Dividing whole Data into Train and Test part with the Ratio of Train to Test is 0.96 : 0.04**

In [ ]:
df.drop('region', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
train,test = train_test_split(df, train_size=0.96, random_state= 2)

**Displaying the Size of Train and Test Part and Resetting to the Default Index of these portions**

In [ ]:
print("Train Portion size is: ",train.shape)
print("Test Portion size is: ",test.shape)

In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

**Defining the function that will be used for Converting the Dataset into Text Data Format so that the the Data can be Tokenize**

In [ ]:
# Identify top 5 cuisines
top_cuisines = df['cuisine'].value_counts().nlargest(5).index.tolist()


In [ ]:
top_cuisines

In [ ]:
top_cuisine_tokens = [f'<CUISINE_{cuisine.upper()}>' for cuisine in top_cuisines]
top_cuisine_tokens

In [ ]:
def df_to_plaintext_file(input_df, output_file, top_cuisines):
    print("Writing to", output_file)
    with open(output_file, 'w', encoding="utf-8") as f:
        for index, row in input_df.iterrows():
            # Skip the recipes that are not in the top 5 cuisines
            if row['cuisine'] not in top_cuisines:
                continue

            # Add the cuisine token for top 5 cuisines
            cuisine_token = f"<CUISINE_{row['cuisine'].upper()}>"
            title = row.title
            instructions = ' '.join(row.instructions).split('.')[:-1]  # Assuming instructions are in a list
            ingredients = row.ingredient_phrase
            keyword = row.ingredients

            # For debugging every 40000 entries
            if index % 40000 == 0:
                print(index)
                print("ingreds --->", ingredients)
                print("keywords --->", keyword)

            # Construct the formatted text
            res = f"{cuisine_token} <RECIPE_START> <INPUT_START> " + " <NEXT_INPUT> ".join(keyword) + \
                  " <INPUT_END> <TITLE_START> " + title + "<TITLE_END> <INGR_START> " + \
                  " <NEXT_INGR> ".join(ingredients) + " <INGR_END> <INSTR_START> " + \
                  " <NEXT_INSTR> ".join(instructions) + " <INSTR_END> <RECIPE_END>"

            f.write(f"{res}\n")

**Saving the Processed Train and Test Files to the Custom Path**

In [ ]:
df_to_plaintext_file(train, '/content/drive/MyDrive/BTP_Dev/Dataset/train_temp.txt',top_cuisines)
df_to_plaintext_file(test, '/content/drive/MyDrive/BTP_Dev/Dataset/test_temp.txt',top_cuisines)

**Initializing the GPT2 Tokenizer and Adding special Tokens defined by us to Define the different parts of the Recipe like its title, constituting ingredeints, etc**

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", do_lower_case=False)
# tokenizer = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
special_tokens = {
    "additional_special_tokens": [
        '<RECIPE_START>',
        '<INPUT_START>',
        '<NEXT_INPUT>',
        '<INPUT_END>',
        '<INGR_START>',
        '<NEXT_INGR>',
        '<INGR_END>',
        '<INSTR_START>',
        '<NEXT_INSTR>',
        '<INSTR_END>',
        '<TITLE_START>',
        '<TITLE_END>',
        '<RECIPE_END>'
    ] + top_cuisine_tokens
}
tokenizer.add_special_tokens(special_tokens)

end_token_id = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]

hf = h5py.File("/content/drive/MyDrive/BTP_Dev/Dataset/data_temp.h5", "w")
for filename in ["/content/drive/MyDrive/BTP_Dev/Dataset/test_temp", "/content/drive/MyDrive/BTP_Dev/Dataset/train_temp"]:
    out_np = []
    data = open(filename+".txt", "r")
    num = 0
    rows = 0
    last=[]

    # Determine if we are processing the training or test set
    is_train = "train" in filename

    # Get the corresponding part of the DataFrame
    current_df = train if is_train else test

    for line, (_, row) in zip(data, current_df.iterrows()):
        num += 1
        if num % 10000 == 0:
            print("Read "+str(num)+" Written: "+str(rows))

        # Add the cuisine token at the start of the line
        if row['cuisine'] in top_cuisines:
          cuisine_token = f"<CUISINE_{row['cuisine'].upper()}>"
          line = f"{cuisine_token} {line}"
        # Tokenization and ID conversion
        text_tokens = tokenizer.tokenize(line)
        if len(text_tokens) > 1024:
            continue
        text_tokens_ids = tokenizer.convert_tokens_to_ids(text_tokens)

        if (len(last) + len(text_tokens_ids)) <= 1024:
            last+=text_tokens_ids
        else:
            while len(last) < 1024:
                last.append(end_token_id)
            out_np.append(last)
            last=text_tokens_ids
            rows+=1
    out_mat = np.matrix(out_np)
    print(out_mat.shape)
    hf.create_dataset(filename, data=out_mat)
hf.close()

**Displaying the Final Length of Tokenizer**

In [ ]:
len(tokenizer)

**Displaying the Final Number of Recipes Downsampled**

In [ ]:
t = []
with open('/content/drive/MyDrive/BTP_Dev/Dataset/train_temp.txt') as file1:
    for f in file1:
        t.append(f)

In [ ]:
print('No of recipes downsampled for prototyping: ',len(t))